#IMPORTING LIBRARIES

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import ElasticNet
from sklearn.decomposition import PCA
# from catboost import CatBoostRegressor
from scipy.sparse import random
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error as sklearn_mean_absolute_error
from sklearn.metrics import mean_squared_error as sklearn_mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from scipy.sparse import random

#TRAIN TEST DATASET

In [ ]:
df = pd.read_csv('kemerer.csv')
df.head(5)

,ID,Language,Hardware,Duration,KSLOC,AdjFP,RAWFP,EffortMM
0,1,1,1,17,253.6,1217.1,1010,287.00
1,2,1,2,7,40.5,507.3,457,82.50
2,3,1,3,15,450.0,2306.8,2284,1107.31
3,4,1,1,18,214.4,788.5,881,86.90
4,5,1,2,13,449.9,1337.6,1583,336.30


In [ ]:
from scipy.sparse import random
X = df.drop(columns=["EffortMM"])
y=df[['EffortMM']]
#X = df[selected_attributes].values
#print (X)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)


# FEATURE SCALING

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        15 non-null     int64  
 1   Language  15 non-null     int64  
 2   Hardware  15 non-null     int64  
 3   Duration  15 non-null     int64  
 4   KSLOC     15 non-null     float64
 5   AdjFP     15 non-null     float64
 6   RAWFP     15 non-null     int64  
 7   EffortMM  15 non-null     float64
dtypes: float64(3), int64(5)
memory usage: 1.1 KB


In [ ]:
df.describe()

,ID,Language,Hardware,Duration,KSLOC,AdjFP,RAWFP,EffortMM
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
mean,8.000000,1.200000,2.333333,14.266667,186.573333,999.140000,993.866667,219.247333
std,4.472136,0.560612,1.676163,7.544787,136.817420,589.592082,597.426130,263.055429
min,1.000000,1.000000,1.000000,5.000000,39.000000,99.900000,97.000000,23.200000
25%,4.500000,1.000000,1.000000,9.000000,55.100000,599.100000,581.000000,83.250000
50%,8.000000,1.000000,2.000000,14.000000,164.800000,993.000000,976.000000,130.300000
75%,11.500000,1.000000,3.500000,17.500000,253.900000,1342.550000,1464.500000,252.800000
max,15.000000,3.000000,6.000000,31.000000,450.000000,2306.800000,2284.000000,1107.310000


#Stacked PCR

In [ ]:
from sklearn.decomposition import PCA


# Apply PCA for dimensionality reduction
n_components = 3  # Adjust the number of components as needed
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Define base regressors
base_regressors = [
    ('linear_regression', LinearRegression()),
    ('svr', SVR(kernel='linear')),
    ('random_forest', RandomForestRegressor(n_estimators=50, random_state=0))
]

# Define the meta-regressor
meta_regressor = LinearRegression()

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_pca, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_pca)

def mean_absolute_error(y_true, y_pred):
    mae = sklearn_mean_absolute_error(y_true, y_pred)
    converted_value = mae /1000
    return converted_value

def mean_squared_error(y_true, y_pred):
    mse = sklearn_mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    converted_value = rmse / 1000
    return converted_value

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Error (MAE): 0.10676985242406484
Root Mean Squared Error (RMSE): 0.14352213196802183
R-squared (R2) Score: -12.990891061509496


#Linear Regression
and hyper parameter tunuing

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)
predict = regressor.predict(X_test)
# Hyperparameter Tuning
param_grid = {
    'fit_intercept': [True, False]

}

grid_search = GridSearchCV(LinearRegression(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Best parameters found
print("Best Parameters:", grid_search.best_params_)

# Evaluating the model with best parameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

# Evaluation Metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

Best Parameters: {'fit_intercept': True}
Mean Absolute Percentage Error (MMRE): 2.2461588946298376
Mean Absolute Error (MAE): 0.1537663921720681
Root Mean Squared Error (RMSE): 0.1837602352626338
R-squared (R2) Score: -21.935631238331197


#Support Vector Machine

In [ ]:
from sklearn.svm import SVR
#X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

support=SVR(kernel='rbf')
support.fit(X_train,y_train)
y_pred=support.predict(X_test)


# Evaluation Metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Absolute Percentage Error (MMRE): 3.123352846523313
Mean Absolute Error (MAE): 0.12073834194793927
Root Mean Squared Error (RMSE): 0.12621165773994253
R-squared (R2) Score: -9.819484350509907


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#Random Forest Regression

In [ ]:

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)
for i in np.arange(100,1100,100):
    random = RandomForestRegressor(n_estimators=i,max_features=5)
    random.fit(X_train, y_train)
    y_pred = random.predict(X_test)

# Evaluation Metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

<ipython-input-10-640209f7f80c>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)
<ipython-input-10-640209f7f80c>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)
<ipython-input-10-640209f7f80c>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)
<ipython-input-10-640209f7f80c>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)
<ipython-input-10-640209f7f80c>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

Mean Absolute Percentage Error (MMRE): 1.8744682884012545
Mean Absolute Error (MAE): 0.09948362333333356
Root Mean Squared Error (RMSE): 0.10423007007247317
R-squared (R2) Score: -6.378934927060854


#KNeighbor


In [ ]:


# Instantiate and fit the KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=3, weights='uniform')
neigh.fit(X_train, y_train)

# Predict on the test data
y_pred = neigh.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Absolute Percentage Error (MMRE): 1.7270085916637639
Mean Absolute Error (MAE): 0.08919999999999999
Root Mean Squared Error (RMSE): 0.10671370837692576
R-squared (R2) Score: -6.734781420888746


#Bagging

In [ ]:
from sklearn.ensemble import BaggingRegressor

# Replace DecisionTreeRegressor with BaggingRegressor
bagging_dtree = BaggingRegressor(base_estimator=DecisionTreeRegressor(random_state=22),
                                 n_estimators=10,  # You can adjust the number of base estimators (trees)
                                 random_state=22)
bagging_dtree.fit(X_train, y_train)
y_pred = bagging_dtree.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

Mean Absolute Percentage Error (MMRE): 1.4065588645071407
Mean Absolute Error (MAE): 0.06657666666666669
Root Mean Squared Error (RMSE): 0.07860889707914749
R-squared (R2) Score: -3.197116526219988


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


#Boosting

In [ ]:

gradient_booster = GradientBoostingRegressor(learning_rate=0.1)
gradient_booster.fit(X_train,y_train)
y_pred = gradient_booster.predict(X_test)


# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Absolute Percentage Error (MMRE): 1.2892270900530425
Mean Absolute Error (MAE): 0.06829249259941998
Root Mean Squared Error (RMSE): 0.09010325828512955
R-squared (R2) Score: -4.514277441619229


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#CATBOOST

In [ ]:

from catboost import CatBoostRegressor

# Instantiate the CatBoostRegressor model
catboost_model = CatBoostRegressor(iterations=100, depth=8, learning_rate=0.1, loss_function='MAE', random_seed=0)

# Fit the model to the training data
catboost_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = catboost_model.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


0:	learn: 80.3154526	total: 46.6ms	remaining: 4.61s
1:	learn: 74.3885434	total: 47.5ms	remaining: 2.33s
2:	learn: 66.4522344	total: 48ms	remaining: 1.55s
3:	learn: 62.4374324	total: 48.4ms	remaining: 1.16s
4:	learn: 58.9368889	total: 48.8ms	remaining: 927ms
5:	learn: 56.1057832	total: 49.2ms	remaining: 771ms
6:	learn: 51.8051179	total: 49.6ms	remaining: 659ms
7:	learn: 48.4700763	total: 50.1ms	remaining: 576ms
8:	learn: 45.2363034	total: 50.4ms	remaining: 510ms
9:	learn: 42.2232471	total: 50.8ms	remaining: 457ms
10:	learn: 39.3122796	total: 51.2ms	remaining: 414ms
11:	learn: 36.9655137	total: 51.5ms	remaining: 378ms
12:	learn: 35.9446384	total: 51.8ms	remaining: 347ms
13:	learn: 34.7142517	total: 52.2ms	remaining: 321ms
14:	learn: 31.0951874	total: 52.6ms	remaining: 298ms
15:	learn: 28.3314097	total: 53.1ms	remaining: 279ms
16:	learn: 25.7636915	total: 53.5ms	remaining: 261ms
17:	learn: 25.6257771	total: 53.9ms	remaining: 246ms
18:	learn: 24.2059035	total: 54.4ms	remaining: 232ms
19:	l

# Stacked Rpart


In [ ]:

# Define base regressors
base_regressors = [
    ('decision_tree', DecisionTreeRegressor(max_depth=5, random_state=0)),
    ('linear_regression', LinearRegression()),
    ('svr', SVR(kernel='linear')),
    ('random_forest', RandomForestRegressor(n_estimators=50, random_state=0))
]

# Define the meta-regressor
meta_regressor = DecisionTreeRegressor()

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Error (MAE): 0.03615025
Root Mean Squared Error (RMSE): 0.05250361911573334
R-squared (R2) Score: -0.42872426682374076


#Stacking using decision tree


In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Define base models
dt1 = DecisionTreeRegressor(max_depth=10, random_state=42)
dt2 = DecisionTreeRegressor(max_depth=5, random_state=42)

# Fit the base models on the training data
dt1.fit(X_train, y_train)
dt2.fit(X_train, y_train)

# Make predictions on the test data using the base models
dt1_y_pred = dt1.predict(X_test)
dt2_y_pred = dt2.predict(X_test)

# Stack the predictions from the base models into a new feature matrix
stacking_train = np.column_stack((dt1.predict(X_train), dt2.predict(X_train)))
stacking_test = np.column_stack((dt1_y_pred, dt2_y_pred))

# Fit a meta model on the new feature matrix and the training labels
meta_model = RandomForestRegressor(n_estimators=100, random_state=42)
meta_model.fit(stacking_train, y_train)

# Make predictions on the test data using the meta model
y_pred = meta_model.predict(stacking_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

Mean Absolute Percentage Error (MMRE): 1.2851130094043892
Mean Absolute Error (MAE): 0.06721800000000011
Root Mean Squared Error (RMSE): 0.08689243171109154
R-squared (R2) Score: -4.128277569585659


<ipython-input-15-f9c3f75d1e71>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  meta_model.fit(stacking_train, y_train)


# **Stacking using SVM**

In [ ]:




# Define base models
svm1 = SVR(kernel='linear')
svm2 = SVR(kernel='rbf')

# Fit the base models on the training data
svm1.fit(X_train, y_train)
svm2.fit(X_train, y_train)

# Make predictions on the test data using the base models
svm1_y_pred = svm1.predict(X_test)
svm2_y_pred = svm2.predict(X_test)

# Stack the predictions from the base models into a new feature matrix
stacking_train = np.column_stack((svm1.predict(X_train), svm2.predict(X_train)))
stacking_test = np.column_stack((svm1_y_pred, svm2_y_pred))

# Fit a meta model on the new feature matrix and the training labels
meta_model = RandomForestRegressor(n_estimators=100, random_state=42)
meta_model.fit(stacking_train, y_train)

# Make predictions on the test data using the meta model
y_pred = meta_model.predict(stacking_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Absolute Percentage Error (MMRE): 1.482105468477883
Mean Absolute Error (MAE): 0.0866801333333334
Root Mean Squared Error (RMSE): 0.11575279203797505
R-squared (R2) Score: -8.10061142842655


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-16-e17561edb8db>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  meta_model.fit(stacking_train, y_train)


# **Stacking using Random Forest**

In [ ]:
# Define base models
rf1 = RandomForestRegressor(n_estimators=10, random_state=40)
rf2 = RandomForestRegressor(n_estimators=50, random_state=10)
rf3 = RandomForestRegressor(n_estimators=50, random_state=42)

# Fit the base models on the training data
rf1.fit(X_train, y_train)
rf2.fit(X_train, y_train)
rf3.fit(X_train, y_train)

# Make predictions on the test data using the base models
rf1_y_pred = rf1.predict(X_test)
rf2_y_pred = rf2.predict(X_test)
rf3_y_pred = rf3.predict(X_test)

# Stack the predictions from the base models into a new feature matrix
stacking_train = np.column_stack((rf1.predict(X_train), rf2.predict(X_train), rf3.predict(X_train)))
stacking_test = np.column_stack((rf1_y_pred, rf2_y_pred, rf3_y_pred))

# Fit a meta model on the new feature matrix and the training labels
meta_model = RandomForestRegressor(n_estimators=500, random_state=42)
meta_model.fit(stacking_train, y_train)

# Make predictions on the test data using the meta model
y_pred = meta_model.predict(stacking_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


<ipython-input-17-1384de0f96aa>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf1.fit(X_train, y_train)
<ipython-input-17-1384de0f96aa>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf2.fit(X_train, y_train)
<ipython-input-17-1384de0f96aa>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf3.fit(X_train, y_train)
<ipython-input-17-1384de0f96aa>:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  meta_model.fit(stacking_train, y_train)


Mean Absolute Percentage Error (MMRE): 1.5393972901428066
Mean Absolute Error (MAE): 0.09114573333333287
Root Mean Squared Error (RMSE): 0.09752782858883498
R-squared (R2) Score: -5.460479174743943


# Stacked XGBoost

In [ ]:
from xgboost import XGBRegressor


# Create the base models (XGBoost)
xgb_model1 = XGBRegressor(n_estimators=100, random_state=42)
xgb_model2 = XGBRegressor(n_estimators=100, random_state=42)

# Train the base models
xgb_model1.fit(X_train, y_train)
xgb_model2.fit(X_train, y_train)

# Make predictions with the base models
xgb_predictions1 = xgb_model1.predict(X_test)
xgb_predictions2 = xgb_model2.predict(X_test)

# Create a meta-model (XGBoost) to combine predictions
meta_model = XGBRegressor(n_estimators=100, random_state=42)

# Combine the predictions from base models as meta-features
meta_features = pd.DataFrame({'XGB1': xgb_predictions1, 'XGB2': xgb_predictions2})

# Train the meta-model on the predictions
meta_model.fit(meta_features, y_test)

# Make predictions with the meta-model
meta_predictions = meta_model.predict(meta_features)

# Calculate the MAE for the ensemble model
mae = mean_absolute_error(y_test, meta_predictions)
mse = mean_squared_error(y_test, meta_predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, meta_predictions)
mmre = mean_absolute_percentage_error(y_test, meta_predictions)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print(f"Ensemble MAE: {mae}")

print(f"Ensemble RMSE: {rmse}")
print(f"Ensemble R-squared: {r2}")


Mean Absolute Percentage Error (MMRE): 2.5416035055916793e-05
Ensemble MAE: 9.679158528648202e-07
Ensemble RMSE: 0.0010314214374955374
Ensemble R-squared: 0.9999999992313082


# Stacked GradientBoost

In [ ]:
# Define base models
base_model1 = GradientBoostingRegressor(n_estimators=100, random_state=42)
base_model2 = GradientBoostingRegressor(n_estimators=200, random_state=42)

# Define the stacked model
stacked_model = StackingRegressor(
    estimators=[
        ('model1', base_model1),
        ('model2', base_model2)
    ],
    final_estimator=GradientBoostingRegressor(n_estimators=50, random_state=42)
)

# Fit the stacked model
stacked_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = stacked_model.predict(X_test)


# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Percentage Error (MMRE): 1.143412906198332
Mean Absolute Error (MAE): 0.05977197875279396
Root Mean Squared Error (RMSE): 0.0760698351971997
R-squared (R2) Score: -2.9303621760226433


#Stacked nnet

In [ ]:
from sklearn.neural_network import MLPRegressor


# Define base regressors (neural networks)
base_regressors = [
    ('mlp_1', MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000, random_state=0)),
    ('mlp_2', MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=0)),
    ('mlp_3', MLPRegressor(hidden_layer_sizes=(50, 25), max_iter=1000, random_state=0))
]

# Define the meta-regressor
meta_regressor = MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000, random_state=0)

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklear

Mean Absolute Percentage Error (MMRE): 2.4222146303341447
Mean Absolute Error (MAE): 0.1456924486983599
Root Mean Squared Error (RMSE): 0.17759156069549836
R-squared (R2) Score: -20.421617911998265


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


#Stacked glmnet

In [ ]:
from sklearn.linear_model import ElasticNet


# Define base regressors (GLMNet - ElasticNet)
base_regressors = [
    ('elastic_net_1', ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=0)),
    ('elastic_net_2', ElasticNet(alpha=0.01, l1_ratio=0.7, random_state=0)),
    ('elastic_net_3', ElasticNet(alpha=0.001, l1_ratio=0.9, random_state=0))
]

# Define the meta-regressor
meta_regressor = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=0)

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):",mmre)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Absolute Percentage Error (MMRE): 2.238543057503877
Mean Absolute Error (MAE): 0.14451563496447647
Root Mean Squared Error (RMSE): 0.1643437429575158
R-squared (R2) Score: -17.344841708330826


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.797e+04, tolerance: 8.538e+01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.407e+04, tolerance: 8.387e+01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-package